In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.float_format', '{:.6f}'.format)
start_time = time.time()

# Import Model

In [2]:
models_path = os.path.join(os.getcwd(),'trained_models')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

0   outputs
1   testing.ipynb
2   trained_models


In [3]:
model = models[8]
model

IndexError: list index out of range

In [ ]:
model_path = os.path.join(models_path,model)

In [ ]:
model_files = os.listdir(model_path)
model_files

In [ ]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [ ]:
with open(spec_path, "r") as specs:
    for line in specs:
        print(line.strip())

In [ ]:
model_path

In [ ]:
model_files

In [ ]:
# insample_path = os.path.join(model_path,[file for file in model_files if file.find('insample') != -1][0])
# insample = pd.read_csv(insample_path).iloc[:,1:]
# insample

In [ ]:
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outsample') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]
outsample

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
plt.figure()
for b in barrier_type_names:
    plt.plot(MAE.loc[:, b], label=b)
plt.xlabel('maturities in days')
plt.ylabel('RMSE')
plt.legend()
plt.show()
plt.close()

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    print(
        f"\n         {'#'*(len(w)+5)}"
        f"\n         # {w}s #"
        f"\n         {'#'*(len(w)+5)}"
    )
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    print(f"RMSE:\n{RMSE}")
    errors['RMSE'] = RMSE
    


    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    print(f"MAE:\n{MAE}")
    errors['MAE'] = MAE

    errors_bypayoff[w] = errors
    print("\n\n")

In [ ]:
barrier_type_names

In [ ]:
ws

In [ ]:
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        print(errors['MAE'])

In [ ]:
plt.figure()
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['MAE'][b]
        plt.plot(
            MAE.sort_values(), label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.show()
plt.close()

In [ ]:
end_time = time.time()
runtime = end_time - start_time
print(f"cpu: {runtime}")